# Equity Analyst Agent with Assistants API

### This notebook showcases the capabilities of Azure OpenAI's Assistants API for an Equity Analyst Agent. 

Install the necessary Python packages (openai, matplotlib, tenacity, python-dotenv) for the notebook to function.

In [1]:
%pip install openai matplotlib tenacity python-dotenv

Note: you may need to restart the kernel to use updated packages.


Imports the necessary Python modules and classes used in the notebook. Note the openai module is used to interact with the Assistants API.

In [2]:
import io
import os
from datetime import datetime
from pathlib import Path
import matplotlib.pyplot as plt
from typing import Iterable
from dotenv import load_dotenv
from openai import AzureOpenAI
from openai.types import FileObject
from openai.types.beta.threads.message_content_image_file import MessageContentImageFile
from openai.types.beta.threads.message_content_text import MessageContentText
from openai.types.beta.threads.messages import MessageFile

### Environment Configuration
This cell is crucial for setting up the environment configuration necessary for the notebook to interact with Azure OpenAI. 

- **Requirement**: Ensure that a `.env` file exists in the same directory as this notebook. This file should contain the necessary API credentials and configuration details, which you can obtain from https://ai.azure.com 
- **Keys in .env File**: The `.env` file must include the following keys:
  - `OPENAI_ENDPOINT`: The endpoint URL for the Azure OpenAI service.
  - `OPENAI_API_KEY`: Your API key for accessing Azure OpenAI services.
  - `OPENAI_MODEL_NAME`: The name of the specific Azure OpenAI model you intend to use.


In [3]:
load_dotenv(".env")
api_endpoint = os.getenv("OPENAI_ENDPOINT")
api_key = os.getenv("OPENAI_API_KEY")
api_deployment_name = os.getenv("OPENAI_MODEL_NAME")
api_version = "2024-02-15-preview"

### Initializing Azure OpenAI Client

Initializes `AzureOpenAI` client with necessary credentials and configurations:
- `api_key`: API key for authentication.
- `api_version`: Targeted API version, set to `"2024-02-15-preview"`.
- `azure_endpoint`: Endpoint URL for Azure OpenAI services.

This step is crucial for establishing communication with Azure OpenAI services.


In [4]:
client = AzureOpenAI(api_key=api_key, 
                     api_version=api_version, 
                     azure_endpoint=api_endpoint)

### Create the assistant with tools

In [5]:
tools_list = [{"type": "code_interpreter"}]

In [12]:
DATASETS = "datasets/"

def upload_file(client: AzureOpenAI, path: Path) -> FileObject:
    with path.open("rb") as f:
        return client.files.create(file=f, purpose="assistants")

assistant_files = [upload_file(client, Path(DATASETS) / file) for file in os.listdir(DATASETS)]
file_ids = [file.id for file in assistant_files]

In [13]:
assistant = client.beta.assistants.create(
    name="Equity Analyst",
    instructions=("You are an equity analyst that performs analysis on the given datasets. "
                  "Use the provided file only."),
    tools=tools_list,
    model=api_deployment_name,
    file_ids=file_ids,
)

thread = client.beta.threads.create()

In [14]:
def format_messages(messages: Iterable[MessageFile]) -> None:

    message_list = []

    # Iterate through the messages and break when a user message is encountered
    for message in messages:
        message_list.append(message)
        if message.role == "user":
            break

    # Reverse the list of messages
    message_list = message_list[::-1]

    for message in message_list:
        for item in message.content:
            if isinstance(item, MessageContentText):
                print(f"{message.role}:\n{item.text.value}\n")
            elif isinstance(item, MessageContentImageFile):
                try:
                    response_content = client.files.content(item.image_file.file_id)
                    data_in_bytes = response_content.read()
                    readable_buffer = io.BytesIO(data_in_bytes)
                    image = plt.imread(readable_buffer, format='jpeg')
                    plt.imshow(image)
                    plt.axis('off')
                    plt.show()
                except Exception as e:
                    print(f"Exception: {e}")

In [15]:
from tenacity import retry, stop_after_attempt, wait_exponential, RetryError, retry_if_exception_type

# Custom exception for specific retry condition
class NotCompletedException(Exception):
    pass

@retry(stop=stop_after_attempt(15), 
       wait=wait_exponential(multiplier=1.5, min=4, max=20),
       retry=retry_if_exception_type(NotCompletedException))
def check_run_status(thread_id, run_id):
    run = client.beta.threads.runs.retrieve(thread_id=thread_id, run_id=run_id)
    if run.status in ["completed", "failed", "expired", "cancelled"]:
        return run
    elif run.status == "requires_action":
        # Handle cases that require action differently
        # For example, you might not want to retry in this case
        pass
    else:
        # This will cause a retry for statuses not explicitly handled above
        raise NotCompletedException("Run not completed yet")

def process_message(content: str):
    client.beta.threads.messages.create(thread_id=thread.id, role="user", content=content)

    run = client.beta.threads.runs.create(
        thread_id=thread.id,
        assistant_id=assistant.id,
        instructions=f"You are a equity analyst who maps out the ask of the user to an equity analyst's task and thinks step by step to analyze, including making use of the tools.",
    )

    try:
        run = check_run_status(thread.id, run.id)
        messages = client.beta.threads.messages.list(thread_id=thread.id)
        format_messages(messages)
    except RetryError:
        print("Operation failed or timed out after maximum retries.")
    except NotCompletedException:
        print("Operation did not complete in the expected status.")


### Have the model perform a DCF valuation

In [17]:
process_message("Create and execute python code to perform a discounted cash flow valuation using the given data in csv. Make generic assumptions")

user:
Create and execute python code to perform a discounted cash flow valuation using the given data in csv. Make generic assumptions

assistant:
Certainly, to perform a discounted cash flow valuation using the given CSV data and generic assumptions, let's outline the steps and code needed to calculate the DCF:

1. We will use the financial projections provided in the CSV data (already loaded).
2. We will assume a terminal growth rate after 2028.
3. We will assume a market-based weighted average cost of capital (WACC) for the discount rate.
4. Calculate the terminal value using the perpetuity growth model.
5. Discount the future cash flows and the terminal value to the present value using the WACC.
6. Sum all the discounted future cash flows and the discounted terminal value to get the enterprise value.
7. Subtract the net debt to get the equity value.

For the assumptions, we will use:
- Terminal growth rate: 2.5% (a common assumption for mature companies)
- WACC: 8% (a common assump

### Delete the thread and assistant

In [11]:
for entity in [(client.beta.assistants, assistant), (client.beta.threads, thread)]:
    entity[0].delete(entity[1].id)

for file in assistant_files:
    client.files.delete(file.id)